In [24]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from torchvision import utils
import os

In [25]:
batch_size = 128
image_size = 64
latent_dimension = 100
number_of_epochs = 200
learning_rate = 0.002
beta_01 = 0.5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
transform = transforms.Compose([
    transforms.Resize(size=image_size),
    transforms.CenterCrop(size=image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]) # Normalizing to [-1, 1]
])
dataset_path = "./dataset/training-GAN/Test/"
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [27]:
class Generator(torch.nn.Module):
    def __init__(self, latent_dimension):
        super(Generator, self).__init__()
        
        # First layer
        self.conv_transpose_2d_01 = torch.nn.ConvTranspose2d(
            in_channels=latent_dimension, out_channels=512, kernel_size=4,
            stride=1, padding=0, bias=False
        )
        self.batch_norm_2d_01 = torch.nn.BatchNorm2d(num_features=512)
        self.relu_01 = torch.nn.ReLU(inplace=True)
        
        # Second layer
        self.conv_transpose_2d_02 = torch.nn.ConvTranspose2d(
            in_channels=512, out_channels=256, kernel_size=4,
            stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_02 = torch.nn.BatchNorm2d(num_features=256)
        self.relu_02 = torch.nn.ReLU(inplace=True)
        
        # Third layer
        self.conv_transpose_2d_03 = torch.nn.ConvTranspose2d(
            in_channels=256, out_channels=128, kernel_size=4,
            stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_03 = torch.nn.BatchNorm2d(num_features=128)
        self.relu_03 = torch.nn.ReLU(inplace=True)
        
        # Fourth layer
        self.conv_transpose_2d_04 = torch.nn.ConvTranspose2d(
            in_channels=128, out_channels=64, kernel_size=4,
            stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_04 = torch.nn.BatchNorm2d(num_features=64)
        self.relu_04 = torch.nn.ReLU(inplace=True)
        
        # Fifth layer
        self.conv_transpose_2d_05 = torch.nn.ConvTranspose2d(
            in_channels=64, out_channels=3, kernel_size=4,
            stride=2, padding=1, bias=False
        )
        self.tanh = torch.nn.Tanh()
    
    def forward(self, x):
        x = self.conv_transpose_2d_01(x)
        x = self.batch_norm_2d_01(x)
        x = self.relu_01(x)
        
        x = self.conv_transpose_2d_02(x)
        x = self.batch_norm_2d_02(x)
        x = self.relu_02(x)
        
        x = self.conv_transpose_2d_03(x)
        x = self.batch_norm_2d_03(x)
        x = self.relu_03(x)
        
        x = self.conv_transpose_2d_04(x)
        x = self.batch_norm_2d_04(x)
        x = self.relu_04(x)
        
        x = self.conv_transpose_2d_05(x)
        x = self.tanh(x)
        
        return x

In [28]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        # First layer
        self.conv2d_01 = torch.nn.Conv2d(
            in_channels=3, out_channels=64, kernel_size=4, stride=2, padding=1, bias=False
        )
        self.leaky_relu_01 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=True)
        
        # Second layer
        self.conv2d_02 = torch.nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_02 = torch.nn.BatchNorm2d(num_features=128)
        self.leaky_relu_02 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=True)
        
        # Third layer
        self.conv2d_03 = torch.nn.Conv2d(
            in_channels=128, out_channels=256, kernel_size=4, stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_03 = torch.nn.BatchNorm2d(num_features=256)
        self.leaky_relu_03 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=True)
        
        # Fourth layer
        self.conv2d_04 = torch.nn.Conv2d(
            in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1, bias=False
        )
        self.batch_norm_2d_04 = torch.nn.BatchNorm2d(num_features=512)
        self.leaky_relu_04 = torch.nn.LeakyReLU(negative_slope=0.2, inplace=True)
        
        # Fivth layer
        self.conv2d_05 = torch.nn.Conv2d(
            in_channels=512, out_channels=1, kernel_size=4, stride=1, padding=0, bias=False
        )
        self.sigmoid_05 = torch.nn.Sigmoid()
        
    def forward(self, x):
        x = self.conv2d_01(x)
        x = self.leaky_relu_01(x)
        
        x = self.conv2d_02(x)
        x = self.batch_norm_2d_02(x)
        x = self.leaky_relu_02(x)
        
        x = self.conv2d_03(x)
        x = self.batch_norm_2d_03(x)
        x = self.leaky_relu_03(x)
        
        x = self.conv2d_04(x)
        x = self.batch_norm_2d_04(x)
        x = self.leaky_relu_04(x)
        
        x = self.conv2d_05(x)
        x = self.sigmoid_05(x)
        
        return x.view(-1)

In [29]:
generator_model = Generator(latent_dimension=latent_dimension).to(device)
print(f"Generator: {generator_model}\n")

discriminator_model = Discriminator().to(device)
print(f"Discriminator: {discriminator_model}")

criterion = torch.nn.BCELoss() # Binary cross entropy loss/ BCE
optimizer_generator = torch.optim.Adam(generator_model.parameters(), lr=learning_rate, betas=(beta_01, 0.999))
optimizer_discriminator = torch.optim.Adam(discriminator_model.parameters(), lr=learning_rate, betas=(beta_01, 0.999))

Generator: Generator(
  (conv_transpose_2d_01): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (batch_norm_2d_01): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_01): ReLU(inplace=True)
  (conv_transpose_2d_02): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch_norm_2d_02): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_02): ReLU(inplace=True)
  (conv_transpose_2d_03): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch_norm_2d_03): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_03): ReLU(inplace=True)
  (conv_transpose_2d_04): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (batch_norm_2d_04): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu_04): ReLU(inplace=Tr